In [1]:
from pydub import AudioSegment
import time, datetime
import pandas as pd
#import xlrd         # for importing Excel spreadsheets
import numpy as np
import os
from IPython.display import display, Audio

In [2]:
os.chdir("/Users/mclaugh/Desktop/home/")

!wget http://www.stephenmclaughlin.net/hipstas/wgbh_temp/cpb-aacip-15-24jm6d99.mp3

--2016-10-21 00:49:02--  http://www.stephenmclaughlin.net/hipstas/wgbh_temp/cpb-aacip-15-24jm6d99.mp3
Resolving www.stephenmclaughlin.net... 208.113.163.135
Connecting to www.stephenmclaughlin.net|208.113.163.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42599065 (41M) [audio/mpeg]
Saving to: 'cpb-aacip-15-24jm6d99.mp3.4'

cpb-aacip-15-24jm6d 100%[===================>]  40.62M  3.31MB/s    in 11s     

2016-10-21 00:49:13 (3.84 MB/s) - 'cpb-aacip-15-24jm6d99.mp3.4' saved [42599065/42599065]



In [3]:
tag_data = pd.read_csv("audio-ml-demo-code/Metadata/AAPB_ARLO_400hrs_161007.csv")

tag_data.head()

,AAPB Unique Identifier (GUID),Type,Value,Timecode IN,Timecode OUT,Duration,dur_hh,dur_mm,dur_ss,seq_duration_SS,seq_duration_hh_mm_ss,SonyCi ID,Filename
0,cpb-aacip/500-j9609t48,Significant speaker voice,"Kennedy, John F.",00:00:05,00:01:50,00:28:40,0.0,28.0,40.0,1720,00:28:40,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48.mp3
1,cpb-aacip/500-j9609t48,Sound quality,audience clapping,00:02:11,00:02:20,NaN,NaN,NaN,NaN,1720,00:28:40,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48.mp3
2,cpb-aacip/500-j9609t48,Significant speaker voice,"Kennedy, John F.",00:01:59,00:04:14,NaN,NaN,NaN,NaN,1720,00:28:40,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48.mp3
3,cpb-aacip/500-j9609t48,Sound quality,audience clapping,00:04:15,00:04:22,NaN,NaN,NaN,NaN,1720,00:28:40,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48.mp3
4,cpb-aacip/500-ff3m1h34,Significant speaker voice,"Kennedy, John F.",00:05:58,00:07:16,00:29:47,0.0,29.0,47.0,3507,00:58:27,d440faa5f295427c962e6edc4d0031a8,cpb-aacip-500-ff3m1h34.mp3


In [4]:
# Picking an arbitrary row (which we already downloaded above)
    
row=tag_data.iloc[555]


In [5]:
## Loading mp3 audio, with timer

import time
start_time = time.time()


song = AudioSegment.from_mp3(row['Filename'])


print("%s" % (time.time() - start_time)+" seconds")

40.9582481384 seconds


In [6]:
start = row['Timecode IN']

end = row['Timecode OUT']

a = time.strptime(start, "%H:%M:%S")

b = time.strptime(end, "%H:%M:%S")

start_sec = (60**2 * a.tm_hour) + (60 * a.tm_min) + a.tm_sec

end_sec = (60**2 * b.tm_hour) + (60 * b.tm_min) + b.tm_sec

print start_sec
print end_sec

6
3099


In [7]:
clip_length_sec = end_sec-start_sec

clip_length_msec = clip_length_sec * 1000

start_pos_sec = start_sec

start_pos_msec = start_pos_sec * 1000.0
            
clip_data = song[start_pos_msec:start_pos_msec+clip_length_msec] # Slice notation with brackets
                                                                 # excerpts from AudioSegment 
                                                                 # object, with millisecond 
                                                                 # integers as indices

In [8]:
print row['Value']

King, Martin Luther


In [9]:
## Generate a filename and export audio to WAV file, with timer

start_time = time.time()


clip_filename=row['Value']+"_"+str(start_pos_sec)+"_"+str(clip_length_sec)+"s_"+row['Filename'].replace(".mp4",".wav").replace(".mp3",".wav")

clip_data.export(clip_filename, format="wav", parameters=["-ar 44100", "-acodec pcm_s16le"])


print("%s" % (time.time() - start_time)+" seconds")


5.2020740509 seconds


In [10]:
## Play audio in browser (fairly slow)
#display(Audio(data = clip_filename, rate = 44100))

In [11]:
# Tying it all together

## Passed a row of tag metadata, this function extracts the corresponding audio and writes
## a WAV file.

def create_tag_excerpt(row):
    try:
        song = AudioSegment.from_mp3(row['Filename'])
    except:
        print "ERROR: "+row['Filename']+" can't be found."

    start = row['Timecode IN']
    end = row['Timecode OUT']

    a = time.strptime(start, "%H:%M:%S")
    b = time.strptime(end, "%H:%M:%S")

    start_sec = (60**2 * a.tm_hour) + (60 * a.tm_min) + a.tm_sec
    end_sec = (60**2 * b.tm_hour) + (60 * b.tm_min) + b.tm_sec

    clip_length_sec = end_sec-start_sec
    clip_length_msec = clip_length_sec * 1000
    start_pos_sec = start_sec
    start_pos_msec = start_pos_sec * 1000.0      
    clip_data = song[start_pos_msec:start_pos_msec+clip_length_msec]

    clip_filename=row['Value']+"_"+str(start_pos_sec)+"_"+str(clip_length_sec)+"s_"+row['Filename'].replace(".mp3","")+".wav"
    clip_data=clip_data.set_channels(1)
    clip_data.export(clip_filename, format="wav", parameters=["-ar 44100", "-acodec pcm_s16le"])




In [12]:
## Test on a single file, with timer

start_time = time.time()


tag_data = pd.read_csv("audio-ml-demo-code/Metadata/AAPB_ARLO_400hrs_161007.csv")

os.chdir("/Users/mclaugh/Desktop/home/")

## Running our function on a file in the pandas dataframe 'tag_data'

create_tag_excerpt(tag_data.iloc[555])


print("%s" % (time.time() - start_time)+" seconds")


74.8138010502 seconds


In [ ]:
## Extracts tagged segments from every mp3 file in the current directory and converts to WAV

tag_data = pd.read_csv("/Users/mclaugh/Desktop/home/AAPB_ARLO_400hrs.csv")

os.chdir("/Users/mclaugh/Desktop/WGBH_Files/")

for i in range(len(tag_data)):
    row = tag_data.iloc[i]
    try: 
        if ".mp3" in row['Filename']:
            create_tag_excerpt(row)
    except: print row